# Termination Conditions

### Question: What is a termination condition?
Answer: When agents are running in a team, a Termination Condition defines the criteria for halting a conversation or process. 

It is crucial for ensuring that runs are terminated at the right time, rather than running endlessly.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    temperature=1,
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback for every message. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

In [ ]:
max_msg_termination = MaxMessageTermination(max_messages=3)
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=max_msg_termination)

await Console(round_robin_team.run_stream(task="Describe a futuristic city in 50 words"))

# Combining Termination Conditions in Autogen AgentChat

In Autogen AgentChat, termination conditions can be combined using the logical `AND (&)` and `OR (|)` operators. This allows for more complex termination logic to control when a conversation should stop.

## Example: Combining Termination Conditions  

In the following example, we create a team where the conversation stops if **either** of the following conditions is met:  
1. **A maximum of 10 messages** have been exchanged.  
2. **The critic agent approves a message** by responding with `"APPROVE"`.

By using the `|` (OR) operator, the conversation will terminate as soon as **one of these conditions is satisfied**.


In [ ]:

# Define termination conditions
max_msg_termination = MaxMessageTermination(max_messages=10)
critic_approval_termination = TextMentionTermination(text="APPROVE")

# Combine conditions using OR (|)
combined_termination = max_msg_termination | critic_approval_termination

# Create a round-robin team with the combined termination condition
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=combined_termination)

# Execute the conversation task
await Console(round_robin_team.run_stream(task="Provide a summary of the latest advancements in AI."))


## Using AND (`&`) for Combined Termination Conditions in Autogen AgentChat  

In addition to using the OR (`|`) operator, termination conditions can also be combined using the AND (`&`) operator. This ensures that the conversation will **only stop when all specified conditions are met**.

It can be written like
`combined_termination = max_msg_termination & text_termination`


> You can also create custom termination conditions and details can be found here https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/termination.html